In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os,time
import pymongo

In [2]:
class Anime:
    def __init__(self, url, headers, anime_name):
        self.url = url  # 網址屬性
        self.headers = headers  # headers屬性
        self.anime_name=anime_name  #動畫名字
    def anime_per_rate(self):
        global anime_percentage
        while True:   
            r = requests.get(self.url, headers=self.headers)     
            if r.status_code == 200:
                soup = BeautifulSoup(r.text, 'html.parser')
                anime_rate=soup.find_all('div',class_='scored-line')
                anime_percentage=[]
                for anr in anime_rate:
                    anime_percentage.append(int(anr['style'][7:len(anr['style'])-2]))#獲取各評價之比例
                star=5
                for i in anime_percentage:
                    print('{}星比例: {}%'.format(star,i))
                    star-=1
                break
            else:
                print(f'請求失敗：{r.status_code}')          
    def anime_upload(self):
        while True:                     
            r = requests.get(self.url, headers=self.headers)
            if r.status_code == 200:
                soup = BeautifulSoup(r.text, 'html.parser')
                anime_uploadtime=soup.find_all('p')#找尋所有被<p></p>包住的文字
                for upt in anime_uploadtime:
                    if upt.text.strip()[0]=='上':
                        return upt.text.strip()[5:]#回傳上傳時間
                    else:
                        continue
            else:
                print(f'請求失敗：{r.status_code}')
    def anime_type(self):
        global l
        while True:
            r = requests.get(self.url, headers=self.headers)   
            if r.status_code == 200:
                soup = BeautifulSoup(r.text, 'html.parser')
                anime_type=soup.select_one('.data_type').text.strip()#獲取動畫相關資訊及類型
                l=[]
                l.append(anime_type.split('\n'))
                for i in range(5):
                    print(l[0][i][:4]+': '+l[0][i][4:])
                break
            else:
                print(f'請求失敗：{r.status_code}')
    def anime_view(self):
        while True:
            r = requests.get(self.url, headers=self.headers)
            if r.status_code == 200:
                soup = BeautifulSoup(r.text, 'html.parser')
                anime_viewcount=soup.select_one('.newanime-count').text.strip()#獲取單集觀看次數
                return anime_viewcount[15:]
            else:
                print(f'請求失敗：{r.status_code}')
    def anime_score(self):
        global anime_per_score
        global anime_score_people     
        while True:
            r = requests.get(self.url, headers=self.headers)
            if r.status_code == 200:
                soup = BeautifulSoup(r.text, 'html.parser')
                anime_per_score =float(soup.select_one('.score-overall-number').text.strip())#獲取分數
                anime_score_people=soup.select_one('.score-overall-people').text.strip()#獲取評價人數
                print('評分: {} {}'.format(anime_per_score,anime_score_people))
                break
            else:
                print(f'請求失敗：{r.status_code}')
    def anime_all_episode(self):
        client=pymongo.MongoClient(host="127.0.0.1",port=27017)
        dbname='testDB{}'.format(time.strftime("%Y%m%d"))
        db=client[dbname]
        r = requests.get(self.url, headers=self.headers)
        if r.status_code == 200:
            soup = BeautifulSoup(r.text, 'html.parser')
            anime_season = soup.find_all('li')#找到所有li的class
            anime_per_episode=[]
            for ans in anime_season:
                try:
                    int(ans.text.strip()[0])#判斷第一個字是不是數字(集數)
                    anime_per_episode.append(ans.text.strip())#是的話就把那一段文字加入list
                except:
                    continue

            anime_episodesn=[]
            anime_sn=soup.find_all('a')#找到所有a的class
            for te in anime_sn:
                if te.get('href')[0]=='?':#判斷第一個字是不是問號
                    anime_episodesn.append(te.get('href')[-5:])#是的話把後面的sn編碼放到list中
        else:
            print(f'請求失敗：{r.status_code}')
        ep=[]
        for i in range(len(anime_episodesn)):#印出每集的觀看次數
            print('第{}集 '.format(anime_per_episode[i]), end='')
            self.url='https://ani.gamer.com.tw/animeVideo.php?sn='+anime_episodesn[i]
            print('觀看次數: {} 上架時間: {}'.format(self.anime_view(),self.anime_upload()))
            ep.append(['第{}集 '.format(anime_per_episode[i]),self.anime_view(),self.anime_upload()])
        path_ = r'C:/DataEngineering'
        if not os.path.exists(path_) :
            os.mkdir(path_)
        path_ = r'C:/DataEngineering/{}'.format(time.strftime("%Y%m%d"))
        if not os.path.exists(path_) :
            os.mkdir(path_)
        epp=pd.DataFrame(ep,columns=['集數','觀看次數','上架時間'])
        epp.to_csv(path_+'/{}.csv'.format(self.anime_name),sep=';',encoding='utf-8-sig',index=False)
        try:
            db.create_collection(anime_name)
        except Exception as e:
            print(e)
        if epp.to_dict('records')!=[]:
            db[self.anime_name].insert_many(epp.to_dict('records'))
def main():
    client=pymongo.MongoClient(host="127.0.0.1",port=27017)
    dbname='testDB{}'.format(time.strftime("%Y%m%d"))
    db=client[dbname]
    for d in db.list_collection_names():#嘗試清空該資料庫所有collection，確保在現在查的資料是最新的
        try:
            db[d].drop()
        except:
            continue
    df = pd.DataFrame(columns=['動畫名稱',
                        '總觀看人數',
                        '評分',
                        '評價人數',
                        '5星比例',
                        '4星比例',
                        '3星比例',
                        '2星比例',      
                        '1星比例',
                        '作品類型',
                        '對象族群',
                        '導演監督',
                        '台灣代理',
                        '製作廠商',
                        '最新一集連結',
                        '最新一集首播日期',
                        '封面圖'])
    global anime_per_score
    global anime_score_people
    global l
    global anime_percentage
    global anime_name
    #本網站會擋機器人，所以要帶 headers
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"}

    # 對"巴哈姆特動畫瘋"送出請求
    r = requests.get('https://ani.gamer.com.tw/', headers=headers)
    if r.status_code == 200:
        print(f'請求成功：{r.status_code}')
        # 藉由 BeautifulSoup 套件將網頁原始碼使用 `html.parser` 解析器來解析
        soup = BeautifulSoup(r.text, 'html.parser')
        newanime_item = soup.select_one('.timeline-ver > .newanime-block')#抓取 class 有包含 timeline-ver 元素底下一層 class 有包含 newanime-block 的元素
        anime_items = newanime_item.select('.newanime-date-area:not(.premium-block)')# 將"付費比例"區塊排除
        for anime_item in anime_items:
            anime_name = anime_item.select_one('.anime-name > p').text.strip()
            print('動畫名稱:',anime_name)# 動畫名稱
            anime_watch_number = anime_item.select_one('.anime-watch-number > p').text.strip()
            print('總觀看人數:',anime_watch_number)# 觀看人數
            anime_href = anime_item.select_one('a.anime-card-block').get('href')
            print('最新一集連結:','https://ani.gamer.com.tw/'+anime_href)# 觀看連結
            
            anime=Anime('https://ani.gamer.com.tw/'+anime_href,headers,anime_name)
            anime.anime_score()#分數
            anime.anime_per_rate()#分數比例
            anime.anime_type()#動畫類別
            anime.anime_all_episode()#每集觀看次數

            anime_date = anime_item.select_one('.anime-date-info').contents[-1].string.strip()
            anime_time = anime_item.select_one('.anime-hours').text.strip()
            print('最新一集首播日期:',anime_date, anime_time)  # 日期與時間
            anime_img = anime_item.select_one('img.lazyload').get('src')
            print("封面圖:",anime_img)  # 動畫縮圖
            print('----------')
            df=df.append({'動畫名稱' : anime_name , '總觀看人數' : anime_watch_number,'評分' : anime_per_score,
                          '評價人數' : anime_score_people,'5星比例':anime_percentage[0],'4星比例':anime_percentage[1],
                          '3星比例':anime_percentage[2],'2星比例':anime_percentage[3],'1星比例':anime_percentage[4],
                          '作品類型':l[0][0][4:],'對象族群':l[0][1][4:],'導演監督':l[0][2][4:],
                          '台灣代理':l[0][3][4:],'製作廠商':l[0][4][4:], '最新一集連結' : ('https://ani.gamer.com.tw/'+anime_href),
                          "最新一集首播日期":(anime_date+anime_time),"封面圖":anime_img} , ignore_index=True)
    else:
        print(f'請求失敗：{r.status_code}')
    try:
        db.create_collection('animes')
    except Exception as e:
        print(e)
    db['animes'].insert_many(df.to_dict('records'))
    df.to_csv(r'C:/DataEngineering/{}/anime.csv'.format(time.strftime("%Y%m%d")),sep=';',encoding='utf-8-sig',index=False)
main()

請求成功：200
動畫名稱: 佐佐木與宮野
總觀看人數: 統計中
最新一集連結: https://ani.gamer.com.tw/animeVideo.php?sn=27383
評分: 4.6 32人評價
5星比例: 88%
4星比例: 0%
3星比例: 4%
2星比例: 4%
1星比例: 7%
作品類型: 青春校園
對象族群: 少女
導演監督: 石平信司
台灣代理: 羚邦
製作廠商: Studio DEEN
最新一集首播日期: 01/10 (一) 00:00
封面圖: https://p2.bahamut.com.tw/B/2KU/53/73f8619cb56154c5000fbe0a791exj15.JPG
----------
動畫名稱: 鬼滅之刃 遊郭篇
總觀看人數: 235.5萬
最新一集連結: https://ani.gamer.com.tw/animeVideo.php?sn=27340
評分: 4.9 6,585人評價
5星比例: 94%
4星比例: 3%
3星比例: 1%
2星比例: 1%
1星比例: 3%
作品類型: 奇幻冒險
對象族群: 少年
導演監督: 外崎春雄
台灣代理: 木棉花
製作廠商: ufotable
第1集 觀看次數: 63.8萬 上架時間: 2021/12/05 23:45
第2集 觀看次數: 46.9萬 上架時間: 2021/12/12 23:45
第3集 觀看次數: 43萬 上架時間: 2021/12/19 23:45
第4集 觀看次數: 41.8萬 上架時間: 2021/12/26 23:45
第5集 觀看次數: 39.7萬 上架時間: 2022/01/02 23:45
第6集 觀看次數: 統計中 上架時間: 2022/01/09 23:45
最新一集首播日期: 01/09 (日) 23:45
封面圖: https://p2.bahamut.com.tw/B/2KU/65/cd7a3a8e84eff89744a1e4805d1exjd5.JPG
----------
動畫名稱: Futsal Boys！！！！！
總觀看人數: 統計中
最新一集連結: https://ani.gamer.com.tw/animeVideo.php?sn=27384
評分: 3.7 15人評價
5星比例: 47%
4星比例: 14%
3星比例

5星比例: 65%
4星比例: 13%
3星比例: 6%
2星比例: 3%
1星比例: 15%
作品類型: 其他
對象族群: 少年
導演監督: 助川勇太
台灣代理: 羚邦
製作廠商: 白組
第1集 觀看次數: 4.6萬 上架時間: 2021/04/03 06:30
第2集 觀看次數: 1.6萬 上架時間: 2021/04/10 06:30
第3集 觀看次數: 1.2萬 上架時間: 2021/04/17 06:30
第4集 觀看次數: 8589 上架時間: 2021/04/24 06:30
第5集 觀看次數: 7175 上架時間: 2021/05/01 06:30
第6集 觀看次數: 6273 上架時間: 2021/05/08 06:30
第7集 觀看次數: 5854 上架時間: 2021/05/15 06:30
第8集 觀看次數: 5472 上架時間: 2021/05/22 06:30
第9集 觀看次數: 4784 上架時間: 2021/05/29 06:30
第10集 觀看次數: 4492 上架時間: 2021/06/05 06:30
第11集 觀看次數: 4352 上架時間: 2021/06/12 06:30
第12集 觀看次數: 3827 上架時間: 2021/06/19 06:30
第13集 觀看次數: 4070 上架時間: 2021/06/26 06:30
第14集 觀看次數: 3764 上架時間: 2021/07/03 06:30
第15集 觀看次數: 3380 上架時間: 2021/07/10 06:30
第16集 觀看次數: 3222 上架時間: 2021/07/17 06:30
第17集 觀看次數: 2859 上架時間: 2021/07/24 06:30
第18集 觀看次數: 2594 上架時間: 2021/07/31 06:30
第19集 觀看次數: 2642 上架時間: 2021/08/07 06:30
第20集 觀看次數: 2411 上架時間: 2021/08/14 06:30
第21集 觀看次數: 2650 上架時間: 2021/08/21 06:30
第22集 觀看次數: 2281 上架時間: 2021/08/28 06:30
第23集 觀看次數: 2798 上架時間: 2021/09/04 06:30
第24集 觀看次數: 2187 上

第10集 觀看次數: 5.1萬 上架時間: 2022/01/07 01:25
最新一集首播日期: 01/07 (五) 01:25
封面圖: https://p2.bahamut.com.tw/B/2KU/53/cc8e8975728f32a4f56f441ab41ewu15.JPG
----------
動畫名稱: 最遊記 RELOAD -ZEROIN-
總觀看人數: 1.2萬
最新一集連結: https://ani.gamer.com.tw/animeVideo.php?sn=27360
評分: 4.7 167人評價
5星比例: 88%
4星比例: 5%
3星比例: 5%
2星比例: 2%
1星比例: 2%
作品類型: 奇幻冒險
對象族群: 少年
導演監督: 高田美里
台灣代理: 木棉花
製作廠商: LIDENFILMS
最新一集首播日期: 01/06 (四) 23:00
封面圖: https://p2.bahamut.com.tw/B/2KU/18/c7baf5a8baed5e137a09ed65381ewni5.JPG
----------
動畫名稱: 東京 24 區
總觀看人數: 2萬
最新一集連結: https://ani.gamer.com.tw/animeVideo.php?sn=27358
評分: 4.0 276人評價
5星比例: 63%
4星比例: 10%
3星比例: 10%
2星比例: 5%
1星比例: 14%
作品類型: 科幻未來
對象族群: 少年
導演監督: 津田尚克
台灣代理: 木棉花
製作廠商: CloverWorks
最新一集首播日期: 01/06 (四) 00:30
封面圖: https://p2.bahamut.com.tw/B/2KU/60/2921b7d439fbb5887421c49d091ewgc5.JPG
----------
動畫名稱: 東方少年
總觀看人數: 1.3萬
最新一集連結: https://ani.gamer.com.tw/animeVideo.php?sn=27446
評分: 4.2 169人評價
5星比例: 64%
4星比例: 16%
3星比例: 11%
2星比例: 4%
1星比例: 8%
作品類型: 奇幻冒險
對象族群: 少年
導演監督: 柳澤哲也
台灣代理: 羚邦
製作廠商: A.C.G.T
最新一集

第24集 觀看次數: 1566 上架時間: 2021/12/22 00:00
第25集 觀看次數: 1133 上架時間: 2021/12/29 00:00
最新一集首播日期: 12/29 (三) 00:00
封面圖: https://p2.bahamut.com.tw/B/2KU/06/990017022d343b0af6935e8ceb1eu0q5.JPG
----------
動畫名稱: 境界戰機
總觀看人數: 33.8萬
最新一集連結: https://ani.gamer.com.tw/animeVideo.php?sn=27442
評分: 3.9 702人評價
5星比例: 52%
4星比例: 16%
3星比例: 12%
2星比例: 8%
1星比例: 13%
作品類型: 科幻未來
對象族群: 少年
導演監督: 羽原信義
台灣代理: 曼迪
製作廠商: BANDAI SPIRITS、SUNRISE BEYOND
第1集 觀看次數: 6.2萬 上架時間: 2021/10/05 01:30
第2集 觀看次數: 4.1萬 上架時間: 2021/10/12 01:30
第3集 觀看次數: 3.5萬 上架時間: 2021/10/19 01:30
第4集 觀看次數: 3.1萬 上架時間: 2021/10/26 01:30
第5集 觀看次數: 2.6萬 上架時間: 2021/11/02 01:30
第6集 觀看次數: 2.3萬 上架時間: 2021/11/09 01:30
第7集 觀看次數: 2.1萬 上架時間: 2021/11/16 01:30
第8集 觀看次數: 1.8萬 上架時間: 2021/11/23 01:30
第9集 觀看次數: 1.7萬 上架時間: 2021/11/30 02:00
第10集 觀看次數: 1.7萬 上架時間: 2021/12/07 01:30
第11集 觀看次數: 1.4萬 上架時間: 2021/12/14 01:40
第12集 觀看次數: 1.3萬 上架時間: 2021/12/21 01:30
第13集 觀看次數: 1.3萬 上架時間: 2021/12/28 01:30
最新一集首播日期: 12/28 (二) 01:30
封面圖: https://p2.bahamut.com.tw/B/2KU/38/cade6cad159e86273d65a8b

第7集 觀看次數: 8969 上架時間: 2021/08/26 22:00
第8集 觀看次數: 8861 上架時間: 2021/09/23 22:00
第9集 觀看次數: 7713 上架時間: 2021/10/28 22:00
第10集 觀看次數: 7476 上架時間: 2021/11/25 22:00
第11集 觀看次數: 7154 上架時間: 2021/12/23 22:00
最新一集首播日期: 12/23 (四) 22:00
封面圖: https://p2.bahamut.com.tw/B/2KU/81/116e66adf1cfb375489d3f0e791eru95.JPG
----------
動畫名稱: 魔法科高中的劣等生 追憶篇
總觀看人數: 9.4萬
最新一集連結: https://ani.gamer.com.tw/animeVideo.php?sn=27347
評分: 4.9 2,030人評價
5星比例: 96%
4星比例: 3%
3星比例: 1%
2星比例: 1%
1星比例: 1%
作品類型: 奇幻冒險
對象族群: 少年
導演監督: 吉田りさこ
台灣代理: 曼迪
製作廠商: 8bit
最新一集首播日期: 其他 00:30
封面圖: https://p2.bahamut.com.tw/B/2KU/90/d4dd7a31d1fae3c53a1204e75e1ev5y5.JPG
----------
動畫名稱: 艾梅洛閣下 II 世事件簿 -魔眼蒐集列車 Grace note- 特別篇
總觀看人數: 5.6萬
最新一集連結: https://ani.gamer.com.tw/animeVideo.php?sn=27382
評分: 4.9 1,144人評價
5星比例: 93%
4星比例: 7%
3星比例: 1%
2星比例: 1%
1星比例: 1%
作品類型: 奇幻冒險
對象族群: 少年
導演監督: 加藤誠
台灣代理: 曼迪
製作廠商: TROYCA
最新一集首播日期: 其他 22:30
封面圖: https://p2.bahamut.com.tw/B/2KU/82/868fd73b740ad0aa43843726fa1ev5q5.JPG
----------
動畫名稱: BanG Dream！Episode of Roselia I：約定
總觀看人數: 